In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
keyname = 'Nc'
# keyname = 'An'
# keyname = 'Tr'
# keyname = 'Nc'
path = '/home/dongjc/work/mtd/RNA-seq/' + keyname + '/'

### 1. 生成sample_condition.tsv表

In [3]:
df = pd.read_csv(path + keyname + "_sample.tsv", sep="\t").copy()
df["Time"] = df["Time"].fillna("")
df["Condition"] = (
    df["Strain"]
    + " "
    + df["Strategy"].fillna("")
    + " "
    + df["Carbon Sources"].fillna("")
    + " "
    + df["Temperature"].fillna("")
    + " "
    + df["Time"].fillna("")
)
df

,Sample ID,Strain,Strategy,Carbon Sources,Temperature,Time,Study Title,Series Accession,Related Sample,Condition
0,mt001001,WT,NaN,Barley,34°,,Transcriptomic response to substrate and tempe...,GSE27323,GSM675519,WT Barley 34°
1,mt001002,WT,NaN,Barley,45°,,Transcriptomic response to substrate and tempe...,GSE27323,"GSM675520,GSM675523",WT Barley 45°
2,mt001003,WT,NaN,Glucose,34°,,Transcriptomic response to substrate and tempe...,GSE27323,"GSM675524,GSM675525",WT Glucose 34°
3,mt001004,WT,NaN,Glucose,45°,,Transcriptomic response to substrate and tempe...,GSE27323,GSM675521,WT Glucose 45°
4,mt001005,WT,NaN,Alfalfa,45°,,Transcriptomic response to substrate and tempe...,GSE27323,GSM675522,WT Alfalfa 45°
...,...,...,...,...,...,...,...,...,...,...
103,mt014008,MtLat-1,NaN,Arabinan,45°,4d,The arabinose transporter MtLat-1 is involved ...,GSE221464,"GSM6873690,GSM6873691",MtLat-1 Arabinan 45° 4d
104,mt014009,MtAra-1,NaN,Aarabinose,45°,4h,The arabinose transporter MtLat-1 is involved ...,GSE221464,"GSM6873692,GSM6873693",MtAra-1 Aarabinose 45° 4h
105,mt014010,MtAra-1,NaN,Xylose,45°,4h,The arabinose transporter MtLat-1 is involved ...,GSE221464,"GSM6873694,GSM6873695",MtAra-1 Xylose 45° 4h
106,mt014011,MtAra-1,NaN,Arabinan,45°,4h,The arabinose transporter MtLat-1 is involved ...,GSE221464,"GSM6873696,GSM6873697",MtAra-1 Arabinan 45° 4h


In [4]:
# df保留Sample Number、Sample ID、Series ID、Study Title、Species、Strain、Sample Name、Canbon Sources、Temperature列
df_1 = df[['Sample ID', 'Series Accession','Study Title', 'Condition', 'Related Sample']]

# 如果Conditon最后一位是_，则去掉
df_1.loc[:,'Condition'] = df_1['Condition'].apply(lambda x: x[:-1] if x[-1] == '_' else x)

df_1

,Sample ID,Series Accession,Study Title,Condition,Related Sample
0,mt001001,GSE27323,Transcriptomic response to substrate and tempe...,WT Barley 34°,GSM675519
1,mt001002,GSE27323,Transcriptomic response to substrate and tempe...,WT Barley 45°,"GSM675520,GSM675523"
2,mt001003,GSE27323,Transcriptomic response to substrate and tempe...,WT Glucose 34°,"GSM675524,GSM675525"
3,mt001004,GSE27323,Transcriptomic response to substrate and tempe...,WT Glucose 45°,GSM675521
4,mt001005,GSE27323,Transcriptomic response to substrate and tempe...,WT Alfalfa 45°,GSM675522
...,...,...,...,...,...
103,mt014008,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtLat-1 Arabinan 45° 4d,"GSM6873690,GSM6873691"
104,mt014009,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtAra-1 Aarabinose 45° 4h,"GSM6873692,GSM6873693"
105,mt014010,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtAra-1 Xylose 45° 4h,"GSM6873694,GSM6873695"
106,mt014011,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtAra-1 Arabinan 45° 4h,"GSM6873696,GSM6873697"


In [5]:
# 读取dataset.tsv
df_2 = pd.read_csv(path + keyname + '_dataset.tsv', sep='\t').copy()
# df_2 = pd.read_csv(path + keyname + '_dataset.csv').copy()
df_2 = df_2[['Series Accession', 'PMID']]

# 将df_2的PMID列转换为整数
df_2['PMID'] = df_2['PMID'].fillna(0).astype(int)

df_2

,Series Accession,PMID
0,GSE213575,37013645
1,GSE213573,37013645
2,GSE213570,37013645
3,GSE221464,36966330
4,GSE214002,36691040
5,GSE214142,36478865
6,GSE205182,36165620
7,GSE184074,35257374
8,GSE183387,0
9,GSE114057,31078793


In [6]:
# 将df_1和df_2合并，根据Series ID列
df_3 = pd.merge(df_1, df_2, on='Series Accession', how='left')

# 保存df_3为tsv文件
df_3.to_csv(path + keyname + '_sample_condition.tsv', sep='\t', index=False)

df_3

,Sample ID,Series Accession,Study Title,Condition,Related Sample,PMID
0,mt001001,GSE27323,Transcriptomic response to substrate and tempe...,WT Barley 34°,GSM675519,21964414
1,mt001002,GSE27323,Transcriptomic response to substrate and tempe...,WT Barley 45°,"GSM675520,GSM675523",21964414
2,mt001003,GSE27323,Transcriptomic response to substrate and tempe...,WT Glucose 34°,"GSM675524,GSM675525",21964414
3,mt001004,GSE27323,Transcriptomic response to substrate and tempe...,WT Glucose 45°,GSM675521,21964414
4,mt001005,GSE27323,Transcriptomic response to substrate and tempe...,WT Alfalfa 45°,GSM675522,21964414
...,...,...,...,...,...,...
103,mt014008,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtLat-1 Arabinan 45° 4d,"GSM6873690,GSM6873691",36966330
104,mt014009,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtAra-1 Aarabinose 45° 4h,"GSM6873692,GSM6873693",36966330
105,mt014010,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtAra-1 Xylose 45° 4h,"GSM6873694,GSM6873695",36966330
106,mt014011,GSE221464,The arabinose transporter MtLat-1 is involved ...,MtAra-1 Arabinan 45° 4h,"GSM6873696,GSM6873697",36966330


### 2. 根据基因id获取按文献分类的Exp

In [5]:

# keyname = 'Mt'
# path = '/home/dongjc/work/mtd/RNA-seq/' + keyname + '/'


def get_gene_exp(gene):
    """
    根据基因id，返回该基因在各个文献对应样本的表达量
    """

    df_exp_fname = pd.read_csv(path + keyname + '_exp.tsv', sep='\t')
    df_condition = pd.read_csv(path + keyname + '_sample_condition.tsv', sep='\t')

    # 根据gene id从df_exp_fname中取出对应的行
    df = df_exp_fname[df_exp_fname['Gene id'] == gene].copy()

    # 根据基因ID从exp表中，取出该基因在各个样本的表达量
    df_condition['Sample Exp'] = df_condition['Sample ID'].map(df.set_index('Gene id').iloc[:, 0:].T.to_dict()[gene])

    # df_condition返回的是一个dataframe；构造字典
    output_dict = {
        # 'gene_info': {
        'gene_id': {
            'gene_id': gene
        }
    }
    for _, row in df_condition.iterrows():
        gse = str(row['Series Accession'])
        if gse not in output_dict['gene_id']:
            output_dict['gene_id'][gse] = {
                'Series Accession': gse,
                'Study Title': row['Study Title'],
            }
        sample_number = row['Sample ID']
        output_dict['gene_id'][gse][sample_number] = {
            'Sample ID': sample_number,
            'Related Sample': row['Related Sample'],
            'Condition': row['Condition'],
            'RPKM': row['Sample Exp']
        }
    
    return output_dict

# dict_gene_exp = get_gene_exp('MYCTH_2293947')
# dict_gene_exp = get_gene_exp('ANI_1_1170084')
# dict_gene_exp = get_gene_exp('TRIREDRAFT_73516')
# dict_gene_exp = get_gene_exp('NCU08755')
dict_gene_exp = get_gene_exp('NCU00673')

dict_gene_exp

{'gene_id': {'gene_id': 'NCU00673',
  'GSE35227': {'Series Accession': 'GSE35227',
   'Study Title': 'Conserved and Essential Transcription Factors for Cellulase Gene Expression in Ascomycete Fungi.',
   'nc001001': {'Sample ID': 'nc001001',
    'Related Sample': 'GSM864216',
    'Condition': 'cdr1  Avicel  1h',
    'RPKM': 1746.69},
   'nc001002': {'Sample ID': 'nc001002',
    'Related Sample': 'GSM864217',
    'Condition': 'cdr1  Avicel  4h',
    'RPKM': 1657.92},
   'nc001003': {'Sample ID': 'nc001003',
    'Related Sample': 'GSM864218',
    'Condition': 'cdr1  Sucrose  1h',
    'RPKM': 189.03},
   'nc001004': {'Sample ID': 'nc001004',
    'Related Sample': 'GSM864219',
    'Condition': 'cdr1  Sucrose  4h',
    'RPKM': 139.64},
   'nc001005': {'Sample ID': 'nc001005',
    'Related Sample': 'GSM864220',
    'Condition': 'cdr2  Avicel  1h',
    'RPKM': 2239.92},
   'nc001006': {'Sample ID': 'nc001006',
    'Related Sample': 'GSM864221',
    'Condition': 'cdr2  Avicel  4h',
    'RPKM':

In [6]:
#将output_dict 转为json文件
with open(path + '00673gene_exp.json', 'w') as f:
    json.dump(dict_gene_exp, f, indent=4)